In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import math
import re
import itertools
from numpy import *
from pandas.core.frame import DataFrame 
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import KFold
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import feature_selection
from mrmr import mrmr_classif

In [2]:
def performance(y_predict,y_test):
   #定义RMSE,NRMSE,R2,Pearson,Spearman
   y_predict=y_predict.flatten()
   SS_R=sum((y_test-y_predict)**2)
   SS_T=sum((y_test-np.mean(y_test))**2)
   R2=1-(float(SS_R))/SS_T
   R2=r2_score(y_test,y_predict)
   rmse=np.sqrt(mean_squared_error(y_test, y_predict))
   nrmse=rmse/np.std(y_test)
   pear=pearsonr(y_test, y_predict)[0]
   spear=spearmanr(y_test,y_predict)[0]
   data=[R2,rmse,nrmse,pear,spear]
   return data

In [23]:
def plsmodel(x,y,component):
    ##training set 建立模型
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size = 0.2)
    pls_model_set=PLSRegression(n_components=component,scale=True)###scale=True默认标准化数据
    pls_model=pls_model_set.fit(x_train,y_train)
    y_predict=pls_model.predict(x_train)
    train_outdata=performance(y_predict,y_train)
    ###testing数据集
    y_test_predict=pls_model.predict(x_test)
    test_outdata=performance(y_test_predict,y_test)
    ###合并training和testing结果
    outdata=train_outdata+test_outdata
    return outdata 

In [18]:
def output(x,y,repeat,component):
    ###重复训练次
    outdata=pd.DataFrame(columns=('train_R^2','train_RMSE','train_NRMSE','train_Pear','train_Spea','test_R^2','test_RMSE','test_NRMSE','test_Pear','test_Spea'))
    for i in range(0,repeat):
        outdata.loc[i]=plsmodel(x,y,component)
    out=np.array(outdata.mean(axis=0))
#    outpath="F:/zhulin/1datasz1/RNA/outdata/"
#    outdata.to_csv(os.path.join(outpath,"psl_component{}.csv".format(component)))
    return out

读取数据

In [6]:
all_feature=pd.read_csv('F:/zhulin/1datasz1/RNA/putout/all_x_matrix.csv')
all_feature.head()

,log(fold),AA,AC,AD,AE,AF,AG,AH,AI,AK,...,struc186,struc187,struc188,struc189,struc190,struc191,struc192,struc193,struc194,struc195
0,0.370883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.293804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.111724,0.019355,0.027742,0.013448,2.165161,0.984516,0.727241,1.424194,0.593871,0.555517
2,-0.493495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.881538,0.009286,0.090714,0.007692,1.610714,2.754286,3.296154,1.048571,2.342857,2.788462
3,0.883150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.262368,0.009000,0.008250,0.002105,1.857750,2.976250,2.101053,1.527500,1.989250,1.778421
4,0.582972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
all_x=all_feature.iloc[:,1:]     ## x变量
y=all_feature.iloc[:,0]         #y变量

参考文章的123个feature，使用文章中使用的123个feature

In [24]:
#提取123个feature
feature123=pd.read_csv("F:/zhulin/1datasz1/RNA/features.csv")
feature123=list(feature123['Feature name'])
x_123=all_x[feature123]

#
repeat=100
out123=[]
for i in range(1,11):
    out123.append(output(x_123,y,repeat,i))
out123=pd.DataFrame(out123,columns=('train_R^2','train_RMSE','train_NRMSE','train_Pear','train_Spea','test_R^2','test_RMSE','test_NRMSE','test_Pear','test_Spea'))
out123


,train_R^2,train_RMSE,train_NRMSE,train_Pear,train_Spea,test_R^2,test_RMSE,test_NRMSE,test_Pear,test_Spea
0,0.659013,0.268001,0.583711,0.811711,0.776385,0.350823,0.339334,0.788396,0.711520,0.706078
1,0.804483,0.202728,0.441787,0.896884,0.843550,0.202117,0.360790,0.844072,0.709563,0.628284
2,0.893799,0.148506,0.325145,0.945379,0.923793,0.237471,0.368458,0.840814,0.685979,0.659583
3,0.929665,0.121187,0.264345,0.964174,0.947702,0.052346,0.380839,0.909470,0.681089,0.638358
4,0.955614,0.095655,0.209685,0.977546,0.965343,-0.000580,0.402839,0.916341,0.686822,0.643995
5,0.969438,0.079524,0.173423,0.984593,0.977902,0.080402,0.392417,0.900066,0.688539,0.640319
6,0.977814,0.067191,0.147111,0.988839,0.984235,-0.114949,0.426933,0.965961,0.667538,0.643260
7,0.984296,0.056568,0.123128,0.992113,0.988926,-0.155076,0.429346,0.997174,0.671803,0.646029
8,0.988367,0.048523,0.105946,0.994164,0.991876,-0.427110,0.457549,1.062924,0.663263,0.650686
9,0.991456,0.041313,0.090629,0.995717,0.993791,-0.310237,0.461778,1.055022,0.640539,0.620270


可以看出component=1时，123个feature的预测结果在test_Spea，和R2处最好。

##1：使用700个feature 做spls 模型 component:from 1 to 10

In [25]:
out700=[]
repeat=100
for i in range(1,11):
    out700.append(output(all_x,y,repeat,i))
out700=pd.DataFrame(out700,columns=('train_R^2','train_RMSE','train_NRMSE','train_Pear','train_Spea','test_R^2','test_RMSE','test_NRMSE','test_Pear','test_Spea'))
out700

,train_R^2,train_RMSE,train_NRMSE,train_Pear,train_Spea,test_R^2,test_RMSE,test_NRMSE,test_Pear,test_Spea
0,0.677307,0.260853,0.567730,0.822879,0.798668,0.322609,0.343407,0.800778,0.675987,0.662990
1,0.878921,0.159261,0.347597,0.937489,0.935141,0.307160,0.355736,0.808952,0.682464,0.671127
2,0.941143,0.110940,0.241995,0.970116,0.974176,0.218925,0.361158,0.829732,0.681562,0.666127
3,0.977421,0.068483,0.149787,0.988644,0.984118,0.173979,0.373863,0.848686,0.676118,0.662770
4,0.989736,0.046080,0.100849,0.994854,0.991917,0.121487,0.392113,0.907398,0.631298,0.624485
5,0.995321,0.031274,0.067988,0.997658,0.995217,0.092605,0.384485,0.916923,0.639029,0.637132
6,0.997644,0.021865,0.047999,0.998821,0.997020,-0.010609,0.414230,0.916742,0.649695,0.651642
7,0.998656,0.016605,0.036270,0.999328,0.998119,0.128350,0.384975,0.882876,0.653232,0.643284
8,0.999216,0.012610,0.027508,0.999608,0.998632,0.120508,0.382373,0.883540,0.649430,0.626863
9,0.999568,0.009255,0.020128,0.999784,0.999045,0.124597,0.380753,0.902847,0.645696,0.615907


从结果来看，component=2时，700个feature的取得较好的预测准确率

## feature selection
###1:先删除全部为0的特征  剩余647个feature,并使用647个feature 做pls

In [12]:
mask=(all_x==0).all(0)
col_index=np.where(mask)[0]
aa=np.array(range(0,700))
aa1=np.delete(aa,col_index)
x_647=all_x[all_x.columns[aa1]]
#
repeat=100
out647=[]
for i in range(1,11):
    out647.append(output(x_647,y,repeat,i))
out647=pd.DataFrame(out647,columns=('train_R^2','train_RMSE','train_NRMSE','train_Pear','train_Spea','test_R^2','test_RMSE','test_NRMSE','test_Pear','test_Spea'))
out647

,train_R^2,train_RMSE,train_NRMSE,train_Pear,train_Spea,test_R^2,test_RMSE,test_NRMSE,test_Pear,test_Spea
0,0.681444,0.257763,0.564098,0.825398,0.794560,0.347546,0.355323,0.794113,0.672935,0.668113
1,0.878120,0.160757,0.348565,0.937051,0.934345,0.271602,0.349861,0.824454,0.692523,0.661078
2,0.941049,0.110998,0.242348,0.970070,0.974699,0.324172,0.353911,0.800722,0.684537,0.681593
3,0.977604,0.068176,0.149266,0.988737,0.983920,0.212278,0.373551,0.861585,0.661617,0.633039
4,0.989913,0.045947,0.099952,0.994943,0.992128,-0.093085,0.411335,0.955916,0.628446,0.637034
5,0.995460,0.030812,0.066918,0.997727,0.995162,0.119702,0.389992,0.908298,0.639377,0.623897
6,0.997705,0.021822,0.047438,0.998852,0.997083,0.180170,0.383045,0.879841,0.648109,0.613701
7,0.998717,0.016263,0.035370,0.999358,0.998083,0.248592,0.364489,0.847249,0.681813,0.643505
8,0.999242,0.012434,0.027192,0.999621,0.998601,0.156518,0.388652,0.891915,0.641089,0.617157
9,0.999497,0.010057,0.021916,0.999748,0.998971,-0.057219,0.385320,0.914761,0.662699,0.654608


同理，从 上述结果可以看出component=2时，预测效果最好，且根据相同的训练集和测试集的结果得到，全部为0的feature对结果没有贡献。其647个feature的
结果于700个feature的结果相同

##2：移除掉方差小于0.001的feature，剩余219个feature

In [60]:
var_feature= all_x.var().sort_values()
nn=np.where(var_feature.values>=0.001)
x_219=all_x[var_feature.index[nn]]
#
out219=[]
for i in range(1,11):
    out219.append(output(x_219,y,repeat,i))
out219=pd.DataFrame(out219,columns=('train_R^2','train_RMSE','train_NRMSE','train_Pear','train_Spea','test_R^2','test_RMSE','test_NRMSE','test_Pear','test_Spea'))
out219

,train_R^2,train_RMSE,train_NRMSE,train_Pear,train_Spea,test_R^2,test_RMSE,test_NRMSE,test_Pear,test_Spea
0,0.526688,0.314226,0.687647,0.725456,0.696647,0.211682,0.381506,0.867274,0.590262,0.580270
1,0.640541,0.274602,0.598914,0.800082,0.739707,0.011233,0.407162,0.937702,0.615315,0.582549
2,0.739902,0.235189,0.509572,0.860088,0.838784,-0.456966,0.430022,1.015787,0.577635,0.567941
3,0.783300,0.213127,0.464805,0.884942,0.868915,-0.037967,0.422097,0.972439,0.612678,0.618652
4,0.834247,0.185492,0.406357,0.913305,0.901827,-0.607406,0.519053,1.176235,0.545185,0.597966
5,0.862640,0.169648,0.369496,0.928716,0.908133,-12.178032,0.699246,1.650444,0.521456,0.560221
6,0.888132,0.152984,0.333524,0.942365,0.914912,-1.303877,0.563125,1.289508,0.539651,0.571397
7,0.912327,0.135294,0.294944,0.955124,0.933855,-4.591417,0.676649,1.565352,0.497231,0.535956
8,0.930393,0.120295,0.262772,0.964548,0.943189,-5.036341,0.760949,1.723042,0.466828,0.518946
9,0.947173,0.104649,0.228630,0.973213,0.954028,-2.797323,0.717856,1.655762,0.472281,0.514657


#根据219个feature的结果来看，效果并不好，我们不考虑使用上面的219个feature组合。重新使用647个feature，做forward。

In [ ]:
##对647个feature按照test_Pear的值做forward，其中component=2
repeat=100
out_forward=[]
forward_feature=list(x_647.columns)[0:2]
out_forward.append(output(x_647[forward_feature],y,repeat,2)[8])
all_feature=list(x_647.columns)[2:647]
for j in range(3,647):
    for i in all_feature:
        one_result=0
        if i in forward_feature:
            pass
        else:
            x_forward=x_647[forward_feature+[i]]
            one_forward=output(x_forward,y,repeat,2)[8]
            if one_forward>one_result:
                one_result=one_forward
                one_feature=i
    forward_feature.append(one_feature)
    out_forward.append(one_result) 
    all_feature.remove(i)

In [ ]:
forward_feature_order=pd.DataFrame(forward_feature)
outpath="F:/zhulin/1datasz1/RNA/putout/"
forward_feature_order.to_csv(os.path.join(outpath,"forward_feature_order_split.csv"))

In [ ]:
#根据上面forward 得到的feature order 依次添加feature
out_forward_result=[]
for i in range(3,647):
    x_forward=x_647[forward_feature[0:i]]
    out_forward_result.append(output(x_forward,y,repeat,2))
out_forward_result=pd.DataFrame(out_forward_result,columns=('train_R^2','train_RMSE','train_NRMSE','train_Pear','train_Spea','test_R^2','test_RMSE','test_NRMSE','test_Pear','test_Spea'))
out_forward_result

In [ ]:
outpath="F:/zhulin/1datasz1/RNA/putout/"
out_forward_result.to_csv(os.path.join(outpath,"pls_85sample_647feature_forward_split.csv"))

In [ ]:
col=['#CCCCFF','#FF8080','#00CCFF']
fig=plt.figure(figsize=(10,8))
d=0
for i in ['test_R^2','test_Pear','test_Spea']:
    plt.plot(out_forward_result[i],color=col[d],lw=2, label= '{}'.format(i))
    plt.legend(loc="lower right")
    d+=1
plt.xlim=([3,647])
plt.title('647feature-forward--order-split')
fig.show()